#  Aug 4, 2019 LSTM binary with 3 min keras
* Colab GPU is used in this program to save a time

In [0]:
from __future__ import print_function

In [0]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

Using TensorFlow backend.


# 1. Data
* max_feature = 20000 word based on frequency
* maxlen = maximum length of word after 80 word, default is putting 0

In [0]:
import numpy as np

In [0]:
class Data:
  
  def __init__(self, max_features = 20000, maxlen = 80):
    
     # save np.load
       np_load_old = np.load

     # modify the default parameters of np.load
       np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

      # call load_data with allow_pickle implicitly set to true
       (x_train, y_train), (x_test, y_test)  = imdb.load_data(num_words=10000)

      # restore np.load for future normal usage
       np.load = np_load_old    
    
      # (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) 
      
       x_train = sequence.pad_sequences(x_train, maxlen = maxlen) # setence length 80 and cut after 80 word
       x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
      
       self.x_train,  self.y_train = x_train,  y_train
       self.x_test,   self.y_test  = x_test,   y_test
      
# reference : https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555
# keras IMDB dataset cause a problem so solving it with stackverflow, but pass it. it is not much important
      

# Modeling

In [0]:
class LSTM(models.Model):
  
  def __init__(self, max_features, maxlen):
    x = layers.Input((maxlen,))
    h = layers.Embedding(max_features, 128)(x) # max_feature = 20,000 word, output vector =128, # each word = 128 vector, length of sentence = 80 word -> become 80 x 128
    h = layers.LSTM(128, dropout = 0.2, recurrent_dropout =0.2)(h) 
    y = layers.Dense(1, activation ='sigmoid')(h) 
    super().__init__(x,y)
    
    self.compile(loss ='binary_crossentropy', optimizer ='adam', metrics=['accuracy'])

# Learning and Evaluation

In [0]:
class Machine:
  
  def __init__(self, max_features = 20000, maxlen =80):
      self.data = Data(max_features, maxlen)
      self.model = LSTM(max_features, maxlen)
      
  def run(self, epochs=3, batch_size=32):
    data = self.data
    model = self.model
    print('Training stage')
    print('--------------')
    model.fit(data.x_train, data.y_train,batch_size = batch_size, epochs = epochs, validation_data =(data.x_test, data.y_test))
    
    score, acc = model.evaluate(data.x_test, data.y_test, batch_size = batch_size)
    
    print('Test performance : loss ={0}, acc ={1}'.format(score, acc))

# Running

In [0]:
def main():
    m = Machine()
    m.run()

if __name__ == '__main__':
    main()

W0804 07:22:37.653120 139684550915968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 07:22:37.690060 139684550915968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 07:22:37.697605 139684550915968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 07:22:37.821328 139684550915968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0804 07:22:37.831779 

Training stage
--------------
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 144s 6ms/step - loss: 0.4630 - acc: 0.7801 - val_loss: 0.4086 - val_acc: 0.8187
Epoch 2/3
25000/25000 [==============================] - 140s 6ms/step - loss: 0.3299 - acc: 0.8617 - val_loss: 0.3867 - val_acc: 0.8289
Epoch 3/3
25000/25000 [==============================] - 38s 2ms/step
Test performance : loss =0.4148267342472076, acc =0.83356
